In [2]:
import numpy as np
import pandas as pd

df_customers = pd.read_csv("F:/Kaggle/E-Commerce-Analysis/olist_customers_dataset.csv")
df_sellers = pd.read_csv("F:/Kaggle/E-commerce-Analysis/olist_sellers_dataset.csv")
df_order_reviews = pd.read_csv("F:/kaggle/E-commerce-Analysis/olist_order_reviews_dataset.csv")
df_order_items = pd.read_csv("F:/kaggle/E-commerce-Analysis/olist_order_items_dataset.csv")
df_products = pd.read_csv("F:/kaggle/E-commerce-Analysis/olist_products_dataset.csv")
df_geolocation = pd.read_csv("F:/kaggle/E-commerce-Analysis/olist_geolocation_dataset.csv")
df_product_category_name_translation = pd.read_csv("F:/kaggle/E-commerce-Analysis/product_category_name_translation.csv")
df_orders = pd.read_csv("F:/kaggle/E-commerce-Analysis/olist_orders_dataset.csv")
df_order_payments = pd.read_csv("F:/kaggle/E-commerce-Analysis/olist_order_payments_dataset.csv")

In [5]:
## print the columns of all dataframes

print(
f"'The columns of olist_customers are :-  ',{df_customers.columns} \n \
'The columns of olist_sellers are :-  ',{df_sellers.columns} \n \
'The columns of olist_order_reviews are :-  ',{df_order_reviews.columns} \n \
'The columns of olist_order_items are :-  ',{df_order_items.columns} \n \
'The columns of olist_products are :-  ',{df_products.columns} \n \
'The columns of olist_geolocation are :-  ',{df_geolocation.columns} \n \
'The columns of product_category_name_translation are :-  ',{df_product_category_name_translation.columns} \n \
'The columns of olist_orders_dataset are :-  ',{df_orders.columns} \n \
'The columns of olist_order_payments are :-  ',{df_order_payments.columns}"
)

'The columns of olist_customers are :-  ',Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object') 
 'The columns of olist_sellers are :-  ',Index(['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'], dtype='object') 
 'The columns of olist_order_reviews are :-  ',Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object') 
 'The columns of olist_order_items are :-  ',Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object') 
 'The columns of olist_products are :-  ',Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
  

In [11]:
#1: 計算2016/09/04 - 2018/10/17 月營收and % of Orders purchased in Jan18 with 5 Review score (Use df_orders and df_order_payments)
print(df_orders["order_purchase_timestamp"].max())
print(df_orders["order_purchase_timestamp"].min())

2018-10-17 17:30:18
2016-09-04 21:15:19


In [63]:
#df_orders.info()
df_orders["order_purchase_timestamp"] = pd.to_datetime(df_orders["order_purchase_timestamp"])

df_orders["order_purchase_y"] = df_orders["order_purchase_timestamp"].dt.year
df_orders["order_purchase_m"] = df_orders["order_purchase_timestamp"].dt.month
df_orders["order_purchase_y_m"] = df_orders["order_purchase_y"].astype(str) + "-" +df_orders["order_purchase_m"].astype(str)
df_orders["order_purchase_y_m"] = pd.to_datetime(df_orders["order_purchase_y_m"])
print(df_orders.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  object        
 5   order_delivered_carrier_date   97658 non-null  object        
 6   order_delivered_customer_date  96476 non-null  object        
 7   order_estimated_delivery_date  99441 non-null  object        
 8   order_purchase_y_m             99441 non-null  datetime64[ns]
 9   order_purchase_y               99441 non-null  int64         
 10  order_purchase_m               99441 non-null  int64         
dtypes: datetime64[n

In [70]:
df_orders.groupby(["order_purchase_y_m"]).agg('count').sort_values(by=["order_purchase_y_m"],ascending = True).reset_index()

,order_purchase_y_m,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_purchase_y,order_purchase_m
0,2016-09-01,4,4,4,4,4,2,1,4,4,4
1,2016-10-01,324,324,324,324,318,278,270,324,324,324
2,2016-12-01,1,1,1,1,1,1,1,1,1,1
3,2017-01-01,800,800,800,800,797,766,750,800,800,800
4,2017-02-01,1780,1780,1780,1780,1766,1674,1653,1780,1780,1780
5,2017-03-01,2682,2682,2682,2682,2680,2591,2546,2682,2682,2682
6,2017-04-01,2404,2404,2404,2404,2400,2352,2303,2404,2404,2404
7,2017-05-01,3700,3700,3700,3700,3691,3600,3545,3700,3700,3700
8,2017-06-01,3245,3245,3245,3245,3241,3182,3135,3245,3245,3245
9,2017-07-01,4026,4026,4026,4026,4021,3928,3872,4026,4026,4026
